# Introduction to Pandas

## Setup and preliminaries

We start this notebook by typing a "magic" command that allows iPython notebooks to display plots directly in the browser.

In [ ]:
# Render our plots inline
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In order to read and process files, we are going to use a very powerful, and widely used Python library, called pandas. So, our next step is to import the pandas library in Python, and also import the library matplotlib for generating plots:

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

Pandas should already be installed on your machine, but if you get an error in the import statement above, indicating that pandas is not available, please uncomment the following command and execute:

In [ ]:
# !sudo python3 -m pip install -U pandas

And we type some code to simply change the visual style of the plots. (The code below is optional and not necessary, and for now you do not need to understand what exactly is happening.)

In [ ]:
# Make the graphs a bit bigger
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])

## Getting data
### Basic example: creating a DataFrame from scratch

This is an example of creating a dataframe by passing a list of dictionaries.
- Each dictionary represents a **row** in the data
- The keys are the **column names**, and the values are the **entries** in the row

In [ ]:
df = pd.DataFrame([ 
    {"First Name": "Amy",   "Last Name":" Poehler"},
    {"First Name": "Tina",  "Last Name":" Fey"},
    {"Last Name" : "Fallon"}
])

In [ ]:
df

### More common: creating a DataFrame from a file

More commonly, we import existing data files. Today, we will use a dataset with restaurant inspection results in NYC which is stored at `/data/restaurants.csv`. If you do not have it, then uncomment below the lines that start with ! and execute the following:

In [ ]:
# Creates a folder "data" under the current folder
!mkdir -p data
# Removes any prior file if it exists
!rm -f data/restaurant.csv*
# Fetches the most recent dataset and stores it under the folder data
!curl 'https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD' -o data/restaurant.csv
# Compresses the text file, both for saving space and for loading faster
!gzip data/restaurant.csv
# Take a look at the first 3 lines of the file, to ensure that everything worked fine
!gzip -cd data/restaurant.csv.gz | head -3

### Methods for Reading in files

We want to be able to read and process this file within Python. The pandas library has a very convenient method `read_csv` which reads the file, and returns back a variable that contains its contents. Other variants are:

Command | Purpose
---|---
`pd.read_csv(f)` | Read a CSV file 
`pd.read_json(f)` | Read a JSON file
`pd.read_excel(f)` | Read an Excel file

Note that here, `f` can be a file on your local computer, or a remote URL.

In [ ]:
# Usually, we can apply the read_csv command without any options, and it works reasonably well
restaurants = pd.read_csv("data/restaurant.csv")

In [ ]:
# However, we can also modify the command using options to suit our application, as we will do here:
restaurants = pd.read_csv("data/restaurant.csv", #.gz", 
                          encoding='utf_8',              # Specify the text encoding
                          dtype = 'unicode',             
                          parse_dates = True,            # Try to recognize and convert dates automatically
                          infer_datetime_format = True,
                          low_memory=False)

When you read a CSV, you get back a kind of object called a DataFrame, which is made up of rows and columns. Let's take a look at how the object looks like:

In [ ]:
restaurants.head(2)

The read_csv method has many options, and you can read further in the [online documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.parsers.read_csv.html).

### Accessing Data

You can access columns in a DataFrame the same way you get elements out of a dictionary; the column name is like a key. For example: 

In [ ]:
# Get the borough for every restaurant
restaurants['BORO']

In [ ]:
# If there are no spaces in the column name, we can also use this notation:
restaurants.BORO

You can see the unique values of the column using the `.unique()` method:

In [ ]:
# Get unique values of a column
restaurants.BORO.unique()

### Descriptive statistics

We can see how many rows and columns are in our dataframe using the `shape` attribute:

In [ ]:
# Get total count of (rows, columns)
restaurants.shape      

We can see what columns the dataframe has with the `columns` attribute:

In [ ]:
# List all the columns
restaurants.columns

We can also check the data types for each column using the `dtypes` attribute.

In [ ]:
# List the data type of each column
restaurants.dtypes

The `object` type is a string. For many of these, we would like to change the data types for a few columns, using the `pd.to_numeric` and `pd.to_datetime` functions. We examine how to convert data types below.

Before we move on, note that we can use the method `describe()` to get a quick overview of the data in the dataframe.

In [ ]:
# Summarize the columns
restaurants.describe()

### Descriptive Statistics for Numeric Variables


#### Converting Data Types to Numeric

The `object` type is a string. When we want to convert an object to numeric, we can use the `pd.to_numeric` function, as shown below:

In [ ]:
restaurants["SCORE"] = pd.to_numeric(restaurants["SCORE"])
restaurants.dtypes

#### Basic descriptive statistics for numeric variables

And now that SCORE is a numeric variable, we can get more detailed descriptive statistics for the variable using the `.describe()` command:

In [ ]:
restaurants.SCORE.describe()

And now that SCORE is a numeric variable, we can examine its distribution by using the `hist` command of Pandas, which creates a histogram. (The histogram is also available as `plot.hist`, or `plot(kind='hist'))`.)

In [ ]:
restaurants["SCORE"].hist()

By default, the histogram has ~10 bars in out plot. We can change the resolution of the histogram using the `bins` variable. Larger number of `bins` allow for higher resolution, but if we increase the number too much, many bins end up having very few, or no data points. For example, experiment with changing the balue of bins below, and change the value from 50 to something bigger.

In [ ]:
restaurants["SCORE"].hist(bins=50)

In [ ]:
# A quick exposure to various options of the "hist" command 
restaurants.SCORE.hist(bins=50,       # use 50 bars
                       range=(0,50),  # x-axis from 0 to 50
                       density=True,  # show normalized count (density=True), or raw counts (density= False)
                       figsize=(15,5) # controls the size of the plot
                      )

#### Kernel Density Estimation (KDE)

An alternative to histograms is to use the **kernel density**, which estimates a continuous function, instead of the bucketized counts, which tends to be discontunuous and bumpy. We can access this usind the `.plot(kind='kde')` command:

In [ ]:
# This plots the density of a numeric attribute
# kde = kernel density estimation
restaurants.SCORE.plot(
    kind='kde', 
    color='blue', 
    xlim=(0,50),      # Set the range of the x axis
    figsize=(15,5),    # Set the figure size
    title = "Restaurant Inspection Score Density"
)

### Converting Dates

Now let's convert the dates columns into the appropriate data types. Let's take a look at a few dates.

In [ ]:
restaurants["GRADE DATE"].head(10)

For that, we need to understand first how to [parse dates using the Python conventions](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

The relevant entries from the table are:
* `%m` Month as a zero-padded decimal number.
* `%d` 	Day of the month as a zero-padded decimal number.
* `%Y` Year with century as a decimal number.

Now, we can specify how to parse these dates. Specifically, we use the `to_datetime` method of Pandas, and we specify how the dates are formatted. (In principle, we can let Pandas parse the format automatically, but it is _much_ faster if we specify it ourselves. It is also much less error-prone.)

In [ ]:
restaurants["GRADE DATE"]      = pd.to_datetime(restaurants["GRADE DATE"],      format="%m/%d/%Y")
restaurants["RECORD DATE"]     = pd.to_datetime(restaurants["RECORD DATE"],     format="%m/%d/%Y")
restaurants["INSPECTION DATE"] = pd.to_datetime(restaurants["INSPECTION DATE"], format="%m/%d/%Y")

In [ ]:
# Check that the dates were converted
restaurants.dtypes

In [ ]:
# Summarize the new date columns
restaurants[["INSPECTION DATE", "GRADE DATE", "RECORD DATE"]].describe()

#### Exercise

* Plot a histogram for `INSPECTION DATE` AND `GRADE DATE`. What do you see?
* Try modifying the `bins` parameter. What are the results?
* The `range=(start, finish)` command is often useful, when we want to focus on a particular part of the dataset. Try using that for  `INSPECTION DATE` to limit the dates to be between 1/1/2014 and 05/31/2018.

In [ ]:
# your code here

#### Solution:
<span style="color:white"> 
\# Not very appealing
restaurants['INSPECTION DATE'].hist()
\# Better
restaurants['INSPECTION DATE'].hist(
    range = ('1/1/2014','5/31/2018'), # limit the range of dates, ignore the 1/1/1990 faulty vale
    bins = 53,                        # number of months in the range -- computed manually
    figsize = (15,5)                  # resize 15-width, 5-height
)
restaurants['GRADE DATE'].hist()
restaurants['GRADE DATE'].hist(
    range = ('1/1/2014','5/31/2018'), # limit the range of dates, ignore the 1/1/1990 faulty valye
    bins = 53,                        # number of months in the range -- computed manually
    figsize = (15,5)                  # resize 15-width, 5-height
)

### Categorical Variables

This is less important, but sometimes we want to specify variables to be "Categorical". This is most commonly useful when we have variables that have an implicit order (e.g., the A/B/C grade of the restaurant).

In [ ]:
restaurants["BORO"]                =  pd.Categorical(restaurants["BORO"], ordered=False)
restaurants["GRADE"]               =  pd.Categorical(restaurants["GRADE"], categories = ['A', 'B', 'C'], ordered=True)
restaurants["VIOLATION CODE"]      =  pd.Categorical(restaurants["VIOLATION CODE"], ordered=False)
restaurants["CRITICAL FLAG"]       =  pd.Categorical(restaurants["CRITICAL FLAG"], ordered=False)
restaurants["ACTION"]              =  pd.Categorical(restaurants["ACTION"], ordered=False)
restaurants["CUISINE DESCRIPTION"] =  pd.Categorical(restaurants["CUISINE DESCRIPTION"], ordered=False)
restaurants.dtypes

### Analyzing the content of the columns

We can also get quick statistics about the common values that appear in each column using the `value_counts()` method:

In [ ]:
restaurants["BORO"].value_counts()

In [ ]:
restaurants["DBA"].value_counts()[:10] 

In [ ]:
restaurants["CUISINE DESCRIPTION"].value_counts()[:5]

And we can use the "plot" command to plot the resulting counts. (More details at http://pandas.pydata.org/pandas-docs/stable/visualization.html). Since the data is categorical, we will use a `bar` chart instead of a `histogram`:

In [ ]:
popular = restaurants["CUISINE DESCRIPTION"].value_counts()[:5]
popular.plot(kind ='bar', 
             title = 'Top Cuisine Types')

Hm, that does not look nice. Let's shorten the name of the cuisine for the _"Latin (Cuban, Dominican, Puerto Rican, South & Central American)"_ and replace it with _"Latin American"_. We will use the `replace` command in Pandas:

In [ ]:
restaurants["CUISINE DESCRIPTION"].replace(
    to_replace='Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
    value = 'Latin American',
    inplace=True
)

In [ ]:
restaurants["CUISINE DESCRIPTION"].replace(
    to_replace='CafÃ©/Coffee/Tea',
    value = 'Cafe/Coffee/Tea',
    inplace=True
)

In [ ]:
popular = restaurants["CUISINE DESCRIPTION"].value_counts()[:5]
popular.plot(kind='bar', title = 'Top Cuisine Types')

And, try with violation codes:

In [ ]:
top_violation_counts = restaurants["VIOLATION CODE"].value_counts()[0:10]
top_violation_counts

In [ ]:
top_violation_counts[0:10].plot(kind='bar', 
                            title = 'Most Frequent Violations')  # Specify bar width (0-1) to control gap between bars

#### Exercise

* Create a plot showing the number of inspections that happen across boroughs.

In [ ]:
# your code here

#### Solution 
<span style="color:white">
restaurants['BORO'].value_counts()
\# Same as above; works only for attribute names without spaces
restaurants.BORO.value_counts()
restaurants['BORO'].value_counts().plot(kind='bar', width=.9)
restaurants['BORO'].value_counts().plot(kind='barh', width=.9)

### Selecting a subset of the columns

In a dataframe, we can specify the column(s) that we want to keep, and get back another dataframe with just the subset of the columns that we want to keep.

In [ ]:
column_list = ["INSPECTION DATE","VIOLATION CODE", "DBA", "SCORE"]
restaurants[ column_list ].head(10)

In [ ]:
restaurants[ ["VIOLATION CODE", "VIOLATION DESCRIPTION"] ].head(5)

### Selecting rows

To select rows, we can use the following approach, where we generate a list of boolean values, one for each row of the dataframe, and then we use the list to select which of the rows of the dataframe we want to keep"

In [ ]:
# Create the condition "has mice" for code 04L
mice = (restaurants["VIOLATION CODE"] == "04L")
mice

In [ ]:
# Apply the condition to the dataframe "restaurants" and store the result 
# in a dataframe called  has_mice
has_mice = restaurants[mice]
has_mice

In [ ]:
# List the most frequent DBA values in the dataframe
has_mice["DBA"].value_counts()[:20]

In [ ]:
has_mice["CAMIS"].value_counts()[:5]

In [ ]:
condition = (restaurants["CAMIS"] == '41444879')
restaurants[condition]

And we can use more complex conditions. 

In [ ]:
mice_manhattan =  ( (restaurants["VIOLATION CODE"] == "04L") 
                    & (restaurants["BORO"] == "MANHATTAN") 
                    & (restaurants["ZIPCODE"] == "10012") )

In [ ]:
has_mice_10012 = restaurants[mice_manhattan]
has_mice_10012

In [ ]:
has_mice_10012["DBA"].value_counts()[:20]

In [ ]:
names_mice_10012 = has_mice_10012["DBA"].value_counts()[:10].sort_values(ascending=True)
names_mice_10012.plot(kind='barh', 
                       width=0.9,
                       title='Restaurants in Zip 10012 with Frequent Mice')

### Pivot Tables

[Pivot tables](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) is one of the most commonly used exploratory tools, and in Pandas they are extremely flexible. 

For example, let's try to count the number of restaurants that are inspected every day. 

In [ ]:
pivot = pd.pivot_table(
                        data = restaurants,
                        index   = 'INSPECTION DATE', # specifies the rows
                        values  = 'CAMIS',           # specifies the content of the cells
                        aggfunc = 'count'            # we ask to count how many different CAMIS values we see
                    )

pivot.head()

Now, let's plot this. By default, Pandas considers the "index" column to be the x-axis, and plots the rest.

In [ ]:
pivot.plot(figsize=[15,5],
           legend=None)

#### Deleting rows from a DataFrame

Hm, that '1900-01-01' line is messing things up. Let's delete it, using the `drop` command. Notice a few things:
* We use the `pd.to_datetime` command to convert the '1900-01-01' string into a datetime data type.
* We use the `axis='index'` parameter means that we delete a **row** with that index value. (The `axis='columns'` means that we delete a column.). Often you will see `axis=0` (equivalent to `axis='index'`) and `axis=1` (equivalent to `axis='columns'`).
* The `inplace=True` means that we change directly the dataframe, instead of returning a new dataframe that does not have the deleted value.

In [ ]:
pivot.drop(pd.to_datetime('1900-01-01'), 
           axis='index', 
           inplace=True) 

And let's plot again.

In [ ]:
pivot.plot(title = 'Count of Inspections by Day',
           figsize=[15,5],
           legend=None)

#### Changing date granularity 

We can also use the [resample](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) command to change the frequency from one day, to, say, 7 days. Then we can compute, say, the average (`mean()`) for these days, or the total number (`sum()`) of inspections.

In [ ]:
pivot.resample('7D').mean() # Resamples to every 7 days, and takes the mean over the days

Now, let's plot this. By default, Pandas considers the "index" column to be the x-axis, and plots the rest.

In [ ]:
# Plot the average number of inspections, over 7-day periods
mean_inspections_weekly = pivot.resample('7D').mean()
mean_inspections_weekly.plot(title = 'Count of Inspections by Week',
                                 figsize=[15,5],
                                 legend=None)

In [ ]:
# Plot the total number of inspections, over 1-month periods
mean_inspections_monthly = pivot.resample('1M').sum()
mean_inspections_monthly.plot(title = 'Count of Inspections by Month',
                               figsize=[15,5],
                               legend=None)

In [ ]:
# If we want to modify more aspects of the graph, we will need to store it:
plot = mean_inspections_weekly.plot(figsize=[15,5], legend=None)

# Now, we can tweak the axes and the titles
plot.set_xlabel("Date of Inspection")
plot.set_ylabel("Average Number of Weekly Inspections")
plot.set_title("Number of Inspections over Time")

#### Pivot Table with two (or more) variables)

We would like to break down the results by borough, so we add the `column` parameter.

In [ ]:
pivot2 =  pd.pivot_table(
                            data = restaurants,
                            index = 'INSPECTION DATE', 
                            columns = 'BORO', 
                            values = 'CAMIS', 
                            aggfunc = 'count'
                        )
pivot2.head()

##### Deleting rows and columns

Now, you will notice that there are a few columns and rows that are just noise. The first row with date *'1900-01-01'* is clearly noise, and the *'Missing'* columns the same. We can use the `drop` command of Pandas to drop these.

In [ ]:
# The axis='index' (or axis=0) means that we delete a row with that index value
pivot2 = pivot2.drop(pd.to_datetime('1900-01-01'), 
                     axis='index') 

In [ ]:
# The axis='columns' (or axis=1) means that we delete a columns with that value
pivot2 = pivot2.drop('Missing', 
                     axis='columns')

In [ ]:
# We resample the index, to keep only dates every one month
# For that one month period, we compute the average value
pivot2.resample('1M').mean()

In [ ]:
# If we plot a dataframe then by default the index column
# in our case the INSPECTION DATE becomes the x-axis
# and all the columns become separate lines in the plot
pivot2.resample('1M').mean().plot()

#### Potential Exercise, if we have time: 
Drop the last date, which corresponds to an incomplete month

**Answer**:
<span style='color:white'>
pivot2 = pivot2.resample('1M').mean().drop(pd.to_datetime('2018-07-31', format = "%Y-%m-%d"), 
                     axis='index') 

### (Optional, FYI) Advanced Pivot Tables

We can also add multiple attributes in the index and columns. It is also possible to have multiple aggregation functions, and we can even define our own aggregation functions.

In [ ]:
# We write a function that returns the 
# number of unique items in a list x 
def count_unique(x):
    return len(set(x))

# We break down by BORO and GRADE, and also calculate 
# inspections in unique (unique restaurants) 
# and non-unique entries (effectively, violations)
pivot_advanced = pd.pivot_table(
    data = restaurants, #
    index = 'GRADE DATE', 
    columns = ['BORO', 'GRADE'],
    values = 'CAMIS', 
    aggfunc = ['count', count_unique]
)

In [ ]:
# Take the total number of inspections (unique and non-unique)
agg = pivot_advanced.resample('1M').sum()

# Show the last 5 entries and show the transpose (.T) 
agg.tail().T

### Exercise 1 

Now let's do the same exercise, but instead of counting the number of inspections, we want to compute the average score assigned by the inspectors. Hint: We will need to change the `values` and the `aggfunc` parameters in the `pivot_table` function above.

In [ ]:
pivot_advanced = pd.pivot_table(
    data = restaurants, #
    index = 'GRADE DATE', 
    columns = ['BORO', 'GRADE'],
    values = 'SCORE', 
    aggfunc = 'mean'
)

In [ ]:
# Take the total number of inspections (unique and non-unique)
agg = pivot_advanced.resample('1M').mean()

# Show the last 5 entries and show the transpose (.T) 
agg.tail().T



### Exercise 2

We now want to examine if different cuisines have different inspection scores. Compute the average inspection score by cuisine. Use the `sort_values()` command ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html)) to order cuisines by their inspection scores.

In [ ]:
pivot_advanced = pd.pivot_table(
    data = restaurants, #
    index = 'CUISINE DESCRIPTION', 
    values = 'SCORE', 
    aggfunc = 'mean'
)

In [ ]:
pivot_advanced.sort_values('SCORE')

## Exercise: NYPD Vehicle Collisions

* We interacted with the NYC Restaurant Inspection Data. Now, let's download another dataset, and do some analysis. We will focus on the [NYPD Vehicle Collisions](https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95/data) data set.


### Task 1: 

Download the dataset. Use the "Export" view, get the URL for the CSV file, and dowload it using curl. (See the top of the notebook for guidance.) 


#### Solution

In [ ]:
# It is a big file, ~270Mb. It will take 2-3 minutes to download
!curl https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD -o accidents.csv

In [ ]:
df = pd.read_csv("accidents.csv", low_memory=False)

In [ ]:
df.dtypes


### Task 2: 

Find out the most common contributing factors to the collisions. 
 

#### Solution

In [ ]:
# Task 2: Find out the most common contributing factors to the collisions.
# Notice that we skip the first element (0) of the list, and we get the elements 1:10
df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()[1:10].plot(kind='barh')

### Task 3: 

Break down the number of collisions by borough.





#### Solution

In [ ]:
# Task 3: Break down the number of collisions by borough.
df['BOROUGH'].value_counts().plot(kind='barh')

### Task 4

Find out the how many collisions had 0 persons injured, 1 persons injured, etc. persons injured in each accident. Use the `value_counts()` approach. You may also find the `.plot(logy=True)` option useful when you create the plot to make the y-axis logarigthmic.
 

#### Solution

In [ ]:
# "Chain" style of writing data maniputation operations
plot = (
    df['NUMBER OF PERSONS INJURED'] # We analyze people injured
    .value_counts() # Count how many accidents had this number of injuries
    .sort_index() # Sort the x-axis
    .plot( 
        # kind='bar', # Create a bar plot
        logy=True # Make the y-axis logarithmic
    )
)
plot.set_xlabel("Number of injuries")
plot.set_ylabel("Number of collisions")
plot.set_title("Analysis of number of injuries per collision")

### Task 5

Break down the accidents by borough and contributing factor. Use the `pivot_table` function of Pandas
 

#### Solution

In [ ]:
pivot = pd.pivot_table(
    data = df, # we analyze the df (accidents) dataframe
    index = 'CONTRIBUTING FACTOR VEHICLE 1', 
    columns = 'BOROUGH', 
    values = 'UNIQUE KEY', 
    aggfunc = 'count'
)
pivot

### Task 6

Find the dates with the most accidents. Can you figure out what happened on these days? 


#### Solution

In [ ]:
df.DATE.value_counts()

### Task 7

Plot the number of accidents per day. (Hint: Ensure that your date column is in the right datatype and that it is properly sorted, before plotting)


#### Solution 

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'], format="%m/%d/%Y")

In [ ]:
(
    df.DATE.value_counts() # count the number of accidents per day
    .sort_index() # sort the dates
    .resample('1M') # take periods of 1 month
    .sum() # sum the number of accidents per month
    .drop(pd.to_datetime('2018-05-31'), axis='index') # drop the current month
    .plot() # plot the result
)

### Task 8

Plot the accidents in map. Use a scatter plot using the `plot(kind='scatter', x=..., y=....)` command, and use the `LATITUDE` and `LONGITUDE` parameters. (Hint: **You will have to remove bad data points before getting into the right visual result**. To do this, specify a selection condition to limit the lat/long values to be values that are proper for the NYC region.)

#### Solution

In [ ]:
cleandf = df[(df.LONGITUDE<-50) & (df.LONGITUDE>-74.5) & (df.LATITUDE< 41)]

cleandf.plot(
    kind = 'scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    s = 1, # make the size of the marker 1 pixel
    figsize = (20,15), # increase the size of the figure
    alpha = 0.05 # make each dot to be 95% transparent 
)

### Task 9

Plot the accidents in map as above, but limit the data only to accidents with at least one injury.

#### Solution

In [ ]:
cleandf = df[(df.LONGITUDE<-50) & (df.LONGITUDE>-74.5) & (df.LATITUDE< 41)]

mask_injured = cleandf['NUMBER OF PERSONS INJURED']>=1
injured_df = cleandf[mask_injured]

injured_df.plot(
    kind = 'scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    s = 1, # make the size of the marker 1 pixel
    figsize = (20,15), # increase the size of the figure
    alpha = 0.05 # make each dot to be 95% transparent 
)

### Bonus plots

In [ ]:
# In the code below, we create three plots
# One with  all the accidents (in black)
# One with all accidents that resulted in an injury (with orange)
# One with all the accidents with deaths (with red dots) 
# Then we combine the  plots using the "ax" parameter

df = pd.read_csv("accidents.csv", low_memory=False)

# We put conditions to keep only rows that 
# have reasonable values for LONGITUDE and LATITUDE
# The & character is the "AND" in Pandas
# the df.LATITUDE is equivalent to df['LATITUDE']
cleandf = df[ (df.LATITUDE > 40) & (df.LATITUDE < 41) & (df.LONGITUDE < -72) & (df.LONGITUDE > -74.5) ]


all_accidents_plot = cleandf.plot (
    kind='scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    figsize = (20, 18), # changes the size of the plot to be bigger and square
    color = 'black',
    alpha = 0.05, # makes the data points transparent (1 = opaque, 0 fully transparent)
    s = 0.5 # make the size of the market just 0.5 pixel
)

# We will keep only entries with at least one injury
# note that we cannot use the df.NUMBER OF PERSON INJURED notation
# because the attribute contains spaces.
mask_injured = cleandf['NUMBER OF PERSONS INJURED']>=1
injured_plot = cleandf[mask_injured].plot (
    kind='scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    figsize = (20, 18), # changes the size of the plot to be bigger and square
    alpha = 0.1, # makes the data points transparent (1 = opaque, 0 fully transparent)
    color = 'orange',
    ax = all_accidents_plot, # the ax parameter allows us to combine plots
    s = 1 # make the size of the market just 1 pixel
)

# we want only accidents with at least one death
mask_killed = cleandf['NUMBER OF PERSONS KILLED']>=1
cleandf[mask_killed].plot (
    kind='scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    figsize = (20, 18), # changes the size of the plot to be bigger and square
    alpha = 0.85, # makes the data points transparent (1 = opaque, 0 fully transparent)
    s = 5, # make the size of the market 5 pixels, to be more visible
    color = 'red',
    ax = injured_plot # the ax parameter allows us to combine plots
)

In [ ]:
# Same as above, but with a 2-d density estimation for the location of
# accidents that resulted in at least one death

cleandf = df[ (df.LATITUDE > 40) & (df.LATITUDE < 41) & (df.LONGITUDE < -72) & (df.LONGITUDE > -74.5) ]


all_accidents_plot = cleandf.plot (
    kind='scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    figsize = (20, 18), # changes the size of the plot to be bigger and square
    color = 'black',
    alpha = 0.05, # makes the data points transparent (1 = opaque, 0 fully transparent)
    s = 0.5 # make the size of the market just 0.5 pixel
)

# We will keep only entries with at least one injury
# note that we cannot use the df.NUMBER OF PERSON INJURED notation
# because the attribute contains spaces.
mask_injured = cleandf['NUMBER OF PERSONS INJURED']>=1
injured_plot = cleandf[mask_injured].plot (
    kind='scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    figsize = (20, 18), # changes the size of the plot to be bigger and square
    alpha = 0.1, # makes the data points transparent (1 = opaque, 0 fully transparent)
    color = 'orange',
    ax = all_accidents_plot, # the ax parameter allows us to combine plots
    s = 1 # make the size of the market just 1 pixel
)

# we want only accidents with at least one death
mask_killed = cleandf['NUMBER OF PERSONS KILLED']>=1
killed_plot = cleandf[mask_killed].plot (
    kind='scatter',
    x = 'LONGITUDE',
    y = 'LATITUDE',
    figsize = (20, 18), # changes the size of the plot to be bigger and square
    alpha = 0.85, # makes the data points transparent (1 = opaque, 0 fully transparent)
    s = 5, # make the size of the market 5 pixels, to be more visible
    color = 'red',
    ax = injured_plot # the ax parameter allows us to combine plots
)

sns.set_style("white")
mask = cleandf['NUMBER OF PERSONS KILLED']>=1
sample = cleandf[mask] # .sample(10000)

sns.kdeplot(sample.LONGITUDE, sample.LATITUDE, gridsize=100,
            cmap=plt.cm.BuGn, shade=False, shade_lowest=True, n_levels=20, alpha=0.75, ax = killed_plot)
